# PCA predictions using logistic regression
to show that the added genetic data does not add much to the PCA we need to apply regression on the PCA values to the phenotype and show that the explenability is not too good

perform analysis on phenotypes produced with:
* 0%/10%/30%/50%/70% weight PCA
* 0%/10%/30%/50%/70% heretability

In [1]:
import pandas as pd
import os
import glob
import re
import re
import statsmodels.api as sm
import datetime


In [2]:
GENETICS_PATH = "/sci/nosnap/michall/roeizucker/covar_tests/small_genetics"
COVARS_PATH = "/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK"
PLINK_PATH = "/sci/nosnap/michall/roeizucker/plink2"
TEST_COVAR = "covariates_sex_year_of_birth_AC_batch_40_PC.txt"
COVAR_DF_PREFIX = "temp_covar_df"
VERSION = "multi_effect_score_0.5"

RELATIVE_EFFECT_OF_PC =  0.7347749803596619
# TODO: try to add load json for parameters

In [3]:
covar_df = pd.read_csv(os.path.join(COVARS_PATH,TEST_COVAR),sep="\t",header=None)

In [4]:
import pandas as pd
import glob
import re

# Define the file pattern to match your CSV files.
file_pattern = "temp_covar_df_v=multi_effect_score_*_her=*_cov=*.csv"
file_list = glob.glob(file_pattern)

# Compile a regular expression to extract the numbers from the filename.
pattern = re.compile(
    r"temp_covar_df_v=multi_effect_score_([0-9\.]+)_her=([0-9\.]+)_cov=([0-9\.]+)\.csv"
)

# Dictionary to store DataFrames keyed by (multi_effect_score, her, cov)
dfs = {}

# Loop over each file in the list
for filename in file_list:
    match = pattern.search(filename)
    if match:
        # Extract the numeric values and convert them to floats.
        multi_effect_score = float(match.group(1))
        her = float(match.group(2))
        cov = float(match.group(3))
    else:
        print(f"Filename {filename} did not match the expected pattern.")
        continue  # Skip files that do not match the pattern

    # Read the CSV file into a DataFrame.
    df = pd.read_csv(filename)

    # (Optional) Add columns with the extracted parameter values.
    df["multi_effect_score"] = multi_effect_score
    df["her"] = her
    df["cov"] = cov

    # Use a tuple of parameters as the dictionary key.
    key = (multi_effect_score, her, cov)
    dfs[key] = df

# Example usage: Access the DataFrame for a specific set of parameter values.
# example_key = (0.1, 0.1, 0.1)
# if example_key in dfs:
#     print(f"DataFrame for parameters {example_key}:")
#     print(dfs[example_key].head())
# else:
#     print(f"No DataFrame found for parameters {example_key}")


In [34]:
regression_results = {}

# Loop over each DataFrame in the dictionary.
for key, df in dfs.items():
    multi_effect_score, her, cov = key
    # Check if her + cov is less than 1.
    if her + cov <= 1:
        # print(key)
        # Define predictor column names: PC1, PC2, ..., PC40.
        pc_columns = [f"PC{i}" for i in range(1, 41)]
        # Extract predictors (X) and response (Y).
        temp_df = df.dropna()
        X = temp_df[pc_columns]
        y = temp_df["disease_indicator"]

        # Add an intercept (constant) to the model.
        X = sm.add_constant(X)

        # Fit the linear regression model.
        model = sm.OLS(y, X).fit()

        # Store the fitted model in the results dictionary.
        regression_results[key] = model

# Example: Present the regression summary for each file that met the condition.


In [41]:
# for params, model in regression_results.items():
#     print(f"\nRegression results for parameters (multi_effect_score, her, cov) = {params}:\n")
#     print(model.summary())
#     # break


In [36]:
results = []
columns = ["ver","her","cov","r_sq",'const'] + ["PC" + str(i) for i in range(1,41)] 
for params, model in regression_results.items():
    # if params[0] == 0.2 and params[1] == 0.7:
    # print(f"For parameters (ver, her, cov) = {params}:")
    # print(f"R-squared: {model.rsquared:.3f}")
    # print(f"expected effect: {(RELATIVE_EFFECT_OF_PC * params[2])**2:.6f}")
    # print(model.params)
    ver = params[0]
    her = params[1]
    cov = params[2]
    r_sq = model.rsquared
    coefs = list(model.params)
    
    values = [ver,her,cov,r_sq] + coefs
    results.append(values)
    # print(dir(model))

In [43]:
df = pd.DataFrame(results,columns=columns)

x = str(datetime.datetime.now())
df.to_csv(f"regression_results_{VERSION}_{x}.csv")

In [44]:
df

,ver,her,cov,r_sq,const,PC1,PC2,PC3,PC4,PC5,...,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40
0,0.5,0.3,0.0,0.000330,-123.877078,-0.000456,0.002392,0.002550,-0.001100,0.000181,...,0.000777,0.001015,0.000082,-0.000957,-0.002022,-0.002134,0.000495,0.000042,0.000550,-0.000144
1,0.5,0.1,0.2,0.151217,-71.123904,0.033999,0.033619,0.033777,0.032041,0.031433,...,0.009977,0.008862,0.007056,0.005202,0.005153,0.003287,0.003620,0.003102,0.002345,0.000254
2,0.5,0.5,0.1,0.074967,-159.633452,0.023968,0.024131,0.024824,0.020603,0.022217,...,0.008206,0.006743,0.004135,0.003756,0.002010,0.001025,0.002288,0.002235,0.001556,0.001414
3,0.5,0.1,0.0,0.000194,-71.525519,-0.000619,0.002113,0.000168,-0.001838,0.000206,...,0.000361,0.000721,-0.001176,-0.001798,-0.000400,-0.001194,-0.000040,0.001098,0.000301,-0.000336
4,0.5,0.7,0.2,0.149955,-188.824003,0.033532,0.034493,0.034100,0.028857,0.032071,...,0.011810,0.009015,0.005878,0.005344,0.003147,0.002555,0.003783,0.003997,0.002541,0.001095
5,0.5,0.2,0.3,0.228057,-100.661287,0.041416,0.041018,0.040219,0.037815,0.039076,...,0.012938,0.010645,0.008849,0.007230,0.005801,0.004340,0.004196,0.003603,0.002806,0.001061
6,0.5,0.0,0.3,0.230281,0.457270,0.040139,0.040513,0.039412,0.039929,0.038626,...,0.010588,0.010220,0.008770,0.006910,0.007550,0.004874,0.004054,0.003580,0.002160,0.001265
7,0.5,0.5,0.3,0.227447,-159.425722,0.042355,0.042527,0.042118,0.035532,0.039356,...,0.013399,0.010624,0.007612,0.007302,0.004885,0.003232,0.004223,0.004165,0.003515,0.001921
8,0.5,0.2,0.0,0.000328,-101.154498,-0.001038,0.002334,0.001025,-0.000376,-0.000356,...,0.001815,0.000966,-0.001157,-0.000787,-0.001178,-0.000264,0.000249,0.000609,0.000794,-0.000633
9,0.5,0.7,0.1,0.075123,-188.927123,0.024573,0.025231,0.024042,0.019288,0.022919,...,0.009609,0.007356,0.003876,0.003650,0.001355,0.000660,0.003381,0.002734,0.002433,0.001176


In [46]:
versions = [0.5]
COVARIATE_SCORES = [0,0.1,0.2,0.3,0.5,0.7]
for ver in versions:
    for cov in COVARIATE_SCORES:
        print("ver",ver,"cov",cov,"mean_rseq",df[(df["ver"] == ver) & (df["cov"] == cov)]["r_sq"].mean())

ver 0.5 cov 0 mean_rseq 0.0003629989657909875
ver 0.5 cov 0.1 mean_rseq 0.0758034520697679
ver 0.5 cov 0.2 mean_rseq 0.15121110843336869
ver 0.5 cov 0.3 mean_rseq 0.22837763127303423
ver 0.5 cov 0.5 mean_rseq 0.38293958296792185
ver 0.5 cov 0.7 mean_rseq 0.5370499254340311


In [11]:
conf_matrix = df.groupby(['ver', 'cov'])['r_sq'].mean().unstack('cov')
print(conf_matrix)

cov  0.0  0.1  0.2  0.3  0.5  0.7
ver                              
0.5  NaN  NaN  NaN  NaN  NaN  NaN
